[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/krisograbek/hotdog/blob/main/api/VGG_training.ipynb)
<p align="center">
    <a href="https://colab.research.google.com/github/krisograbek/hotdog/blob/main/api/VGG_training.ipynb">
        <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
    </a>
</p>

# Introduction

The purpose of this notebook is to train an image classification model for the Hot Dog Not Hot Dog dataset. The notebook consists of the following steps:

 - Using kaggle API for downloading
 - Applying Data Augmentation with `ImageDataGenerator`
 - Applying Transfer Learning for the VGG19 model
 - Restoring a model with the lowest validation loss
 - Applying Early stopping [optional]



## Apply kaggle API

We'll use kaggle API in order to download the Hot Dog - Not Hot Dog dataset. [Link to dataset](https://www.kaggle.com/dansbecker/hot-dog-not-hot-dog). We chose to use kaggle because this is a reliable source. Usually, many links for downloading datasets are outdated. On kaggle, datasets stay for years. 

In order to use kaggle API you need a kaggle account with a valid API key. All the steps are explained under [Easiest way to download kaggle data in Google Colab](https://www.kaggle.com/general/74235)

In [1]:
! pip install -q kaggle

Please uncomment the following cell. I commented it because my API Key would be visible in the output of the cell. I wanted to hide it.

In [2]:
# from google.colab import files
# files.upload()

In [3]:
# create a dir for kaggle API
! mkdir ~/.kaggle
# copy the json file with API Key
! cp kaggle.json ~/.kaggle
# make it read-only
! chmod 600 ~/.kaggle/kaggle.json
# download dataset and unzip it locally
! kaggle datasets download -d dansbecker/hot-dog-not-hot-dog -p /content/sample_data/ --unzip

mkdir: cannot create directory ‘/root/.kaggle’: File exists
 93% 83.0M/89.3M [00:00<00:00, 135MB/s]
100% 89.3M/89.3M [00:00<00:00, 140MB/s]


# Code

## Imports

In [4]:
from tensorflow import keras
from keras.applications.vgg19 import VGG19
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.preprocessing.image import ImageDataGenerator

Make the results reproducible

In [5]:
from numpy.random import seed
seed(31)
from tensorflow import random
random.set_seed(21)

## Data Augmentation

The dataset is relatively small. It contains only 498 images in the train set. Applying Data Augmentation is a very convienient way to artifficially increase the number of train images.

In [6]:
# Instantiate two image generator classes:
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    data_format='channels_last',
    rotation_range=30,
    horizontal_flip=True,
    fill_mode='reflect')

valid_datagen = ImageDataGenerator(
    rescale=1.0/255,
    data_format='channels_last')

In [7]:
# Define the batch size:
batch_size=32

# Define the train and validation generators: 
train_generator = train_datagen.flow_from_directory(
    directory='/content/sample_data/train',
    target_size=(224, 224),
    classes=['hot_dog','not_hot_dog'],
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=True,
    seed=42)

valid_generator = valid_datagen.flow_from_directory(
    directory='/content/sample_data/test',
    target_size=(224, 224),
    classes=['hot_dog','not_hot_dog'],
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=True,
    seed=42)

Found 498 images belonging to 2 classes.
Found 500 images belonging to 2 classes.


## Downloading VGG19

In [8]:
vgg19 = VGG19(include_top=False,
              weights='imagenet',
              input_shape=(224,224,3),
              pooling=None)

### Freezing layers

In [9]:
for layer in vgg19.layers:
    layer.trainable = False

## Implementing a Keras Sequential Model

In [10]:
# Instantiate the sequential model and add the VGG19 model: 
model = Sequential()
model.add(vgg19)

# Add the custom layers atop the VGG19 model: 
model.add(Flatten(name='flattened'))
model.add(Dropout(0.5, name='dropout'))
model.add(Dense(2, activation='softmax', name='predictions'))

## Training the Model

In [11]:
# apply early stoping if no improvement for at least 5 steps (patience)
# set restore to True
early_stoping_callback=keras.callbacks.EarlyStopping(
    monitor='val_loss', min_delta=0, patience=5, verbose=2, mode='auto',
    baseline=None, restore_best_weights=True)

checkpoint_filepath = '/content/sample_data/weights.{epoch:02d}.h5'
model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor='val_loss', #  validation loss
    mode='min',         # lowest possible
    save_best_only=True)

# compile model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# train model
history1 = model.fit(train_generator, steps_per_epoch=15, 
                    epochs=15, validation_data=valid_generator, 
                    validation_steps=15, 
                    callbacks=[
                            #    early_stoping_callback, # early stopping 
                               model_checkpoint_callback # only best model 
                               ]
                )

Epoch 1/15
15/15 [==============================] - 32s 1s/step - loss: 1.5638 - accuracy: 0.5258 - val_loss: 1.2757 - val_accuracy: 0.5104
Epoch 2/15
15/15 [==============================] - 13s 892ms/step - loss: 0.9727 - accuracy: 0.6288 - val_loss: 0.7068 - val_accuracy: 0.6771
Epoch 3/15
15/15 [==============================] - 13s 894ms/step - loss: 0.5061 - accuracy: 0.7489 - val_loss: 0.4428 - val_accuracy: 0.7937
Epoch 4/15
15/15 [==============================] - 13s 892ms/step - loss: 0.4157 - accuracy: 0.8026 - val_loss: 0.4300 - val_accuracy: 0.7958
Epoch 5/15
15/15 [==============================] - 13s 893ms/step - loss: 0.4126 - accuracy: 0.8219 - val_loss: 0.5218 - val_accuracy: 0.7563
Epoch 6/15
15/15 [==============================] - 13s 888ms/step - loss: 0.4069 - accuracy: 0.8090 - val_loss: 0.4903 - val_accuracy: 0.7688
Epoch 7/15
15/15 [==============================] - 13s 887ms/step - loss: 0.4226 - accuracy: 0.8133 - val_loss: 0.5667 - val_accuracy: 0.7437
Ep

## Evaluating the Models

### Model after the last epoch

In [12]:
score = model.evaluate(valid_generator, verbose=0)
print(f"Test loss: {score[0]}")
print(f"Test accuracy: {score[1]}")

Test loss: 0.5517483949661255
Test accuracy: 0.7699999809265137


### The best Model
However, the 4th epoch had the lowest validation loss. Let's load the model and compare both models

In [13]:
best_model = keras.models.load_model('/content/sample_data/weights.04.h5')
best_score = best_model.evaluate(valid_generator, verbose=0)
print(f"Test loss: {best_score[0]}")
print(f"Test accuracy: {best_score[1]}")

Test loss: 0.440985769033432
Test accuracy: 0.7919999957084656


The model that we had after 4th epoch performs better. We'll use it in our app.

Please, download the model to `<project dir>/api/models/hotdog_vgg.h5`

# Final Thoughts

In this notebook we showed how to apply transfer learning using a VGG19 model. We achieved around 80% accuracy. It may not seem impressive. Yet, the Hot Dog Dataset is quite small. There are many images that are tricky. If an image contains only a hot dog, our model classifies it correctly with a high probability. Probably, the model could increase accuracy by applying Fine-tuning. It is out of the scope of this notebook.

These steps probably won't bring any value:

- we could increase the number of epochs. However, it's obvious the model starts to overfit. The difference between train and validation metrics increases.
- applying early stopping when there are only 15 epochs isn't beneficial.

Further exploration:

- compare performance without Data Augmentation
- apply Fine-tuning